# REST API Face Samples

## Objective
Utilize face attributes obtained from the face API to assess face image quality in GPT-4 Turbo with Vision

## Time

You should expect to spend 5-10 minutes running this sample.

## Before you begin

#### Installation

In [ ]:
%pip install -r ../requirements.txt

### Parameters
You need to set a series of configurations such as GPT-4V_DEPLOYMENT_NAME, OPENAI_API_BASE, OPENAI_API_VERSION, FACE_API_ENDPOINT.

Add "OPENAI_API_KEY" and "FACE_API_KEY" as variable name and \<Your API Key Value\> and \<Your FACE Key Value\> as variable value in the environment variables.
 <br>
      
      WINDOWS Users: 
         setx OPENAI_API_KEY "REPLACE_WITH_YOUR_KEY_VALUE_HERE"
         setx FACE_API_KEY "REPLACE_WITH_YOUR_KEY_VALUE_HERE"

      MACOS/LINUX Users: 
         export OPENAI_API_KEY="REPLACE_WITH_YOUR_KEY_VALUE_HERE"
         export FACE_API_KEY="REPLACE_WITH_YOUR_KEY_VALUE_HERE"

In [ ]:
# Setting up the deployment name
deployment_name: str = "<your GPT-4 Turbo with Vision deployment name>"
# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base: str = "<your resource base URL>"
# Currently OPENAI API have the following versions available: 2022-12-01.
# All versions follow the YYYY-MM-DD date structure.
openai_api_version: str = "<your OpenAI API version>"

# The base URL for your face resource endpoint, e.g. "https://<your-resource-name>.cognitiveservices.azure.com"
face_api_endpoint: str = "<your face resource endpoint>"

should_cleanup: bool = False

## Connect to your project
To start with let us create a config file with your project details. This file can be used in this sample or other samples to connect to your workspace.

In [ ]:
import json
from pathlib import Path

config = {
    "GPT-4V_DEPLOYMENT_NAME": deployment_name,
    "OPENAI_API_BASE": openai_api_base,
    "OPENAI_API_VERSION": openai_api_version,
    "FACE_API_ENDPOINT": face_api_endpoint,
}

p = Path("../config.json")

with p.open(mode="w") as file:
    file.write(json.dumps(config))

## Run this Example

### Call Face API

In [ ]:
import os
import sys
from PIL import Image

parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from shared_functions import call_face_API

# Setting up the face resource key
face_api_key = os.getenv("FACE_API_KEY")

image_file_path = "portrait_example.jpg"
attributes_from_01, attributes_from_03 = call_face_API(image_file_path, face_api_endpoint, face_api_key)
description = ""
if attributes_from_01 and attributes_from_03 and len(attributes_from_01) > 0 and len(attributes_from_03) > 0:
    with Image.open(image_file_path) as img:
        width, height = img.size
    width, height = img.size
    face_data = attributes_from_03[0]["faceRectangle"]
    face_width, face_height = face_data["width"], face_data["height"]
    face_left, face_top = face_data["left"], face_data["top"]

    # Calculate the center points
    face_center_x = face_left + face_width / 2
    face_center_y = face_top + face_height / 2
    image_center_x, image_center_y = width / 2, height / 2

    face_size_ratio = face_height / height
    center_x_ratio = abs(face_center_x - image_center_x) / width
    center_y_ratio = abs(face_center_y - image_center_y) / height

    if face_size_ratio < 0.45:
        description += "The patient's face takes up less than half (50%) the height of the photograph\n"
    else:
        description += "The patient's face takes up more than half (50%) the height of the photograph\n"
    if center_x_ratio < 0.2 and center_y_ratio < 0.2:
        description += "The patient's face is vertically and horizontally centered in the photograph\n"
    else:
        description += "The patient's face is not centered in the photograph\n"

    faceAttributes = {**attributes_from_01[0]["faceAttributes"], **attributes_from_03[0]["faceAttributes"]}
    for attr, value in faceAttributes.items():
        if isinstance(value, dict):
            description += f"{attr.capitalize()}:\n"
            for subkey, subvalue in value.items():
                description += f"  {subkey.capitalize()}: {subvalue}\n"
        elif isinstance(value, list):
            description += f"{attr.capitalize()}:\n"
            for item in value:
                description += f"  {item}\n"
        else:
            description += f"{attr.capitalize()}: {value}\n"
print(description)

### Call GPT-4 Turbo with Vision API with Image

In [ ]:
import base64
from IPython.display import Image, display
import sys

parent_dir = Path(Path.cwd()).parent
sys.path.append(str(parent_dir))
from shared_functions import call_GPT4V_image

# Image Tagging Assistant
sys_message = """
    Judge the portrait quality by the following 8 criteria. Specifically, please answer yes or no to each criterion and structure the answers of all 8 criteria in a JSON.
    At the end, please provide an overall judgment, either 'Meets Criteria' (should meet all 8 criteria) or 'Does Not Meet Criteria' (if not meet at least one), along with a detailed explanation, 
    based on the responses to the eight criteria.
    1. The camera is level with the patient's face (head pose value < +/-20), resulting in a photograph that is not angled up at the chin or down at the top of the head.
    2. The patient's face is vertically and horizontally centered in the photograph.
    3. The patient's face takes up more than half (50%) the height of the photograph. 
    4. The patient is facing the camera directly with full face in view (head pose value < +/-20) 
    5. The patient is not wearing sunglasses, but reading glasses are okay. A head covering may be worn for cultural, religious or medical purposes. 
    6. The face, including the forehead, eyes, mouth, and nose, is not occluded. The photograph is not grainy, blurry, dark, or exceedingly bright. DO NOT check blur level, only check blur value. The blur value < 0.5 is within the acceptable range for a clear view of the face. 
    7. The photograph appears to have a reasonable uniform or solid background.
    8. To avoid HIPAA violations, there are no other people recognizable in the photograph
    """

# Encode the image in base64
with Path(image_file_path).open("rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

messages = [
    {"role": "system", "content": [{"type": "text", "text": sys_message}]},
    {
        "role": "user",
        "content": [{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}],
    },
    {
        "role": "assistant",
        "content": [{"type": "text", "text": description}],
    },
]

try:
    response_content = call_GPT4V_image(messages, face=True)
    display(Image(image_file_path))
    print(response_content["choices"][0]["message"]["content"])  # Print the content of the response
except Exception as e:
    print(f"Failed to call GPT-4 Turbo with Vision API. Error: {e}")

## Cleaning up

To clean up all Azure ML resources used in this example, you can delete the individual resources you created in this tutorial.

If you made a resource group specifically to run this example, you could instead [delete the resource group](https://learn.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group).

In [ ]:
if should_cleanup:
    # {{TODO: Add resource cleanup}}
    pass